In [156]:
# calcul matriciel
import numpy as np
from sklearn.metrics import accuracy_score
# utilisation des modèles
import tensorflow as tf
# visualisation statistique et images
import matplotlib.pyplot as plt


In [157]:
class MNISTLoader:
  def __init__(self):
    self.x_train = None
    self.y_train = None
    self.x_test = None
    self.y_test = None
    self.x_train_reshaped = None
    self.x_test_reshaped = None
    self.x_test_normalized = None
    self.x_train_normalized = None

  def load_data(self):
    (self.x_train, self.y_train), (self.x_test, self.y_test) = tf.keras.datasets.mnist.load_data()
    self.x_test_reshaped = self.x_test.reshape(-1, 28*28)
    self.x_train_reshaped = self.x_train.reshape(-1, 28*28)
    self.x_test_normalized = self.x_test_reshaped / 255
    self.x_train_normalized = self.x_train_reshaped / 255

  def display_digit_distribution(self):
    # Calculer la distribution des chiffres dans l'ensemble d'apprentissage
    train_counts = np.bincount(self.y_train)
    train_digits = np.nonzero(train_counts)[0]

    # Calculer la distribution des chiffres dans l'ensemble de test
    test_counts = np.bincount(self.y_test)
    test_digits = np.nonzero(test_counts)[0]

    # Tracer les graphiques de distribution
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    ax1.bar(train_digits, train_counts[train_digits])
    ax1.set_title('Distribution des chiffres - Entraînement')
    ax1.set_xlabel('Chiffre')
    ax1.set_ylabel('Nombre d\'occurrences')

    ax2.bar(test_digits, test_counts[test_digits])
    ax2.set_title('Distribution des chiffres - Test')
    ax2.set_xlabel('Chiffre')
    ax2.set_ylabel('Nombre d\'occurrences')

    plt.show()


  def display_image(self, dataset, index):
    if dataset == 'train':
      image = self.x_train[index]
      label = self.y_train[index]
    elif dataset == 'test':
      image = self.x_test[index]
      label = self.y_test[index]
    else:
      raise ValueError('invalid argument dataset')

    plt.imshow(image, cmap='gray')
    plt.title(f'Image {index} - Chiffre: {label}')
    plt.axis('off')
    plt.show()


  def display_digit_means(self):
    digit_means_train = []
    digit_means_test = []

# pour chaque digit on fait la moyenne de la valeur de chaque pixel pour créer une nouvelle image
    for digit in range(10):
    # on prends toutes les images pour lesquelles le label = à l'index (0 sur le premier tour de loop)
      digit_images_train = self.x_train[self.y_train == digit]
      digit_images_test = self.x_test[self.y_test == digit]

      digit_mean_train = np.mean(digit_images_train, axis=0)
      digit_mean_test = np.mean(digit_images_test, axis=0)

      digit_means_train.append(digit_mean_train)
      digit_means_test.append(digit_mean_test)

    # Affichage des moyennes des chiffres
    fig, axs = plt.subplots(2, 10, figsize=(20, 6))

    for i in range(10):
      axs[0, i].imshow(digit_means_train[i], cmap='gray')
      axs[0, i].set_title(f'Moyenne - Chiffre {i}')
      axs[0, i].axis('off')

      axs[1, i].imshow(digit_means_test[i], cmap='gray')
      axs[1, i].set_title(f'Moyenne - Chiffre {i}')
      axs[1, i].axis('off')

    plt.tight_layout()
    plt.show()

In [158]:
loader = MNISTLoader()
loader.load_data()

In [159]:
class my_model:
  def __init__(self):
    # tableau de type (label, mean_value)
    self.mean_label_array = None
    self.y = []
      
  # on transforme toutes les valeurs en 0 ou 1 car ce qui compte c'est pas les nuances mais bien les pixels avec et sans traits
  def binarize(self, x):
    # init an array of size x.size
    res = np.zeros(x.size)
    
    for i in range(x.size):
      if x[i] > 0.4:
        res[i] = 1
      else:
        res[i] = 0
    return res
    
  def train(self, x_dataset, y_dataset):
    unique_labels = np.unique(y_dataset)
    # init a np array of size unique_labels.size
    self.mean_label_array = np.array([])
    
    for i in range(unique_labels.size):
      label_values = x_dataset[y_dataset == unique_labels[i]]
      label_mean = np.mean(label_values, axis=0)
      binarized_label_mean = self.binarize(label_mean)
      print(unique_labels[i])
      self.mean_label_array = np.append(self.mean_label_array, [unique_labels[i], binarized_label_mean])
        
  def predict(self, x):
    self.y = []
    for i in range(x.size):
      binarized_x = self.binarize(x[i])
      self.y[i] = (self.predict_one(binarized_x))
    
    return self.y

  def predict_one(self, x):
    # on initialise la distance minimale à la première valeur du tableau
    print(self.mean_label_array[0])
    min_distance = x - self.mean_label_array[0][1]
    min_label = self.mean_label_array[0][0]
    
    for label, mean in self.mean_label_array:
        distance = x - mean
        if distance < min_distance:
            min_distance = distance
            min_label = label
    
    return min_label
      
      

In [160]:
model = my_model()
model.train(loader.x_train_reshaped, loader.y_train)
y_pred = model.predict(loader.x_test_normalized)

accuracy = accuracy_score(loader.y_test, y_pred)

0


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.